# Lowered & typed code

## An introduction to code transformations in Julia

Tim Holy

JuliaCon 2021

# Multiple ways to write the same code

```julia
if isempty(list)
    idx = nothing
else
    idx = firstindex(list)
end
```

is the same as

```julia
idx = isempty(list) ? nothing : firstindex(list)
```


```julia
function f(x, y)
    return x + y
end
```

is the same as

```julia
f(x, y) = x + y
```

Other examples: `do`-block syntax, `for` vs `while` loops, etc.

# Some higher-level constructs add complexity

```julia
function myfunc(a, b)
    f(z) = z^2 + a      # an inner function
    
    ...
end
```

Scoping rules make the naive parsing of code less straightforward:

In [3]:
function f()
    for i = 1:5
        x = rand()
    end
    println("The last random number was ", x)
end

f()

LoadError: UndefVarError: x not defined

Expressing these with uniform syntax is the main purpose of *lowering*.

# @code_lowered

In [4]:
f(a, b) = a*a + b
@code_lowered f(1, 2)

CodeInfo(
1 ─ %1 = a * a
│   %2 = %1 + b
└──      return %2
)

%n: [Single Static Assignment](https://en.wikipedia.org/wiki/Static_single_assignment_form) value

Left column: [basic block](https://en.wikipedia.org/wiki/Basic_block)

In [5]:
g(cond, val1, val2) = cond ? val1 + val2 : val2
@code_lowered g(true, 1, 2)

CodeInfo(
1 ─      goto #3 if not cond
2 ─ %2 = val1 + val2
└──      return %2
3 ─      return val2
)

In [6]:
function blastoff()
    counter = 5
    while counter > 0
        println(counter)
        counter -= 1
    end
    println("blast off!")
end
@code_lowered blastoff()

CodeInfo(
1 ─      counter = 5
2 ┄ %2 = counter > 0
└──      goto #4 if not %2
3 ─      Main.println(counter)
│        counter = counter - 1
└──      goto #2
4 ─ %7 = Main.println("blast off!")
└──      return %7
)

In [7]:
function mysum(list)
    ss = 0.0
    for item in list
        ss += item
    end
    return ss
end
@code_lowered mysum([1,2,3])

CodeInfo(
1 ─       ss = 0.0
│   %2  = list
│         @_3 = Base.iterate(%2)
│   %4  = @_3 === nothing
│   %5  = Base.not_int(%4)
└──       goto #4 if not %5
2 ┄ %7  = @_3
│         item = Core.getfield(%7, 1)
│   %9  = Core.getfield(%7, 2)
│         ss = ss + item
│         @_3 = Base.iterate(%2, %9)
│   %12 = @_3 === nothing
│   %13 = Base.not_int(%12)
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return ss
)

In [8]:
# We can see scoping in action:
function f()
    for i = 1:5
        x = rand()
    end
    println("The last random number was ", x)
end
@code_lowered f()

CodeInfo(
1 ─ %1  = 1:5
│         @_2 = Base.iterate(%1)
│   %3  = @_2 === nothing
│   %4  = Base.not_int(%3)
└──       goto #4 if not %4
2 ┄ %6  = @_2
│         i = Core.getfield(%6, 1)
│   %8  = Core.getfield(%6, 2)
│         x = Main.rand()
│         @_2 = Base.iterate(%1, %8)
│   %11 = @_2 === nothing
│   %12 = Base.not_int(%11)
└──       goto #4 if not %12
3 ─       goto #2
4 ┄ %15 = Main.println("The last random number was ", Main.x)
└──       return %15
)

# Lowered representation of more complex constructs

An example using inner functions:

In [9]:
function myfunc(a, b)
    f(z) = z^2 + a      # an inner function

    return f(b)
end
@code_lowered myfunc(1, 1)

CodeInfo(
1 ─ %1 = Main.:(var"#f#1")
│   %2 = Core.typeof(a)
│   %3 = Core.apply_type(%1, %2)
│        f = %new(%3, a)
│   %5 = (f)(b)
└──      return %5
)

What is `#f#1`?

Conceptually, Julia expands it like this:

```julia
struct FClosure{A}
    a::A
end
(f::FClosure)(z) = z^2 + f.a

function myfunc(a, b)
    fc = FClosure{typeof(a)}(a)
    return fc(b)
end
```

`#f#1` is the automatically-generated name given to `FClosure`.

We can see this with `Meta.lower(mod, expr)`:

In [10]:
Meta.lower(Main, quote
    function myfunc(a, b)
        f(z) = z^2 + a      # an inner function

        return f(b)
    end
end)

:($(Expr(:thunk, CodeInfo(
    @ In[10]:2 within `top-level scope`
1 ─       $(Expr(:thunk, CodeInfo(
    @ none within `top-level scope`
1 ─     return $(Expr(:method, :myfunc))
)))
│         $(Expr(:method, :myfunc))
│         $(Expr(:thunk, CodeInfo(
    @ none within `top-level scope`
1 ─      global var"#f#2"
│        const var"#f#2"
│        Core.TypeVar(:a, Core.Any)
│   %4 = Core._structtype(Main, Symbol("#f#2"), Core.svec(%3), Core.svec(:a), Core.svec(), false, 1)
│        var"#f#2" = %4
│        Core._setsuper!(var"#f#2", Core.Function)
│        Core._typebody!(var"#f#2", Core.svec(%3))
└──      return nothing
)))
│   %4  = Core.svec(var"#f#2", Core.Any)
│   %5  = Core.svec()
│   %6  = Core.svec(%4, %5, $(QuoteNode(:(#= In[10]:3 =#))))
│         $(Expr(:method, false, :(%6), CodeInfo(
    @ In[10]:3 within `none`
1 ─ %1 = Core.apply_type(Base.Val, 2)
│   %2 = (%1)()
│   %3 = Base.literal_pow(^, z, %2)
│   %4 = Core.getfield(#self#, :a)
│   %5 = %3 + %4
└──      return %5
)))


# @code_typed

This allows you to see the results of type-inference:

In [14]:
f(a, b) = a*a + b
@code_lowered f(1, 2)

CodeInfo(
1 ─ %1 = a * a
│   %2 = %1 + b
└──      return %2
)

In [17]:
@code_typed optimize=false f(1, 2)

CodeInfo(
1 ─ %1 = (a * a)::Int64
│   %2 = (%1 + b)::Int64
└──      return %2
) => Int64

In [18]:
@code_typed optimize=true f(1, 2)

CodeInfo(
1 ─ %1 = Base.mul_int(a, a)::Int64
│   %2 = Base.add_int(%1, b)::Int64
└──      return %2
) => Int64

In [15]:
@code_typed f(1.0, 2)

CodeInfo(
1 ─ %1 = Base.mul_float(a, a)::Float64
│   %2 = Base.sitofp(Float64, b)::Float64
│   %3 = Base.add_float(%1, %2)::Float64
└──      return %3
) => Float64

# Summary

- Each `Method` has a single lowered-code representation
- See it with `@code_lowered` or, in complex cases, `Meta.lower`
- Lowering expands macros, resolves and flattens scope, and reduces control-flow to "goto"s
- `@code_typed` also shows type inference (corresponding to a particular `MethodInstance`)
- `@code_typed` by default applies optimization transformations like inlining and linearization

With practice, these code representations become quite readable